In [687]:
import pandas as pd
import numpy as np
import seaborn as sns


In [688]:
houses_train = pd.read_csv('../data/processed_data.csv',keep_default_na=False)
#houses_test = pd.read_csv('../data/cleaned_test_data.csv',keep_default_na=False)

In [689]:
houses_train.head()

MSSubClass  LotFrontage   LotArea  Street  Utilities  YearBuilt  \
0          60     0.150685  0.033420     1.0        1.0   0.949275   
1          20     0.202055  0.038795     1.0        1.0   0.753623   
2          60     0.160959  0.046507     1.0        1.0   0.934783   
3          70     0.133562  0.038561     1.0        1.0   0.311594   
4          60     0.215753  0.060576     1.0        1.0   0.927536   

   YearRemodAdd  MasVnrArea  BsmtExposure  BsmtUnfSF  ...  Exter_Rating  \
0      0.883333     0.12250          0.25   0.064212  ...      1.166667   
1      0.433333     0.00000          1.00   0.121575  ...      0.833333   
2      0.866667     0.10125          0.50   0.185788  ...      1.166667   
3      0.333333     0.00000          0.25   0.231164  ...      0.833333   
4      0.833333     0.21875          0.75   0.209760  ...      1.166667   

   Bsmt_rating  Basement_bathrooms  Aboveground_bathrooms  Kitchen_Rating  \
0         1.55            0.333333               0.916667        0.222222   
1         1.55            0.250000               0.666667        0.111111   
2         1.55            0.333333               0.916667        0.222222   
3         1.60            0.333333               0.333333        0.222222   
4         1.55            0.333333               0.916667        0.222222   

   Fireplace_Rating  Total_Condtion  BsmtFinType1_rating  BsmtFinType2_rating  \
0          0.000000             2.0             0.125089                  0.0   
1          0.200000             2.0             0.144401                  0.0   
2          0.200000             2.0             0.086109                  0.0   
3          0.266667             2.0             0.031892                  0.0   
4          0.200000             2.0             0.116052                  0.0   

   Bsmt_total_SF_rating  
0              0.125089  
1              0.144401  
2              0.086109  
3              0.031892  
4              0.116052  

[5 rows x 129 columns]

In [690]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [691]:
X = houses_train.drop(['SalePrice'],axis=1)
y = np.log10(houses_train['SalePrice'])

In [692]:
X_train, X_val, y_train, y_val = train_test_split(X.copy(), y.copy(),
                                                                 shuffle = True,
                                                                 random_state = 321,
                                                                 test_size=.25)

In [693]:
number_of_models = 5
model_dict = {}
metrics = ['mse', 'r2_score']


In [694]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train,y_train)
A = model.predict(X_val)

display(mean_squared_error(y_val,A))
display(r2_score(y_val,A))
model_dict['Random_forest'] = [mean_squared_error(y_val,A),r2_score(y_val,A)]

0.0035728399395999412

0.8677733128555142

In [695]:
import xgboost

xgb_reg = xgboost.XGBRegressor(n_estimators=100,
                          max_depth=3,
                          learning_rate=.1)


xgb_reg.fit(X_train, y_train)

B = xgb_reg.predict(X_val)

display(mean_squared_error(y_val,B))
display(r2_score(y_val,B))
model_dict['XGBoost'] = [mean_squared_error(y_val,B),r2_score(y_val,B)]

0.003409051056658917

0.8738349505858662

In [696]:
dic = xgb_reg.get_booster().get_score(importance_type='weight')
keys = list(dic.keys())
values = list(dic.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features




<Axes: >

In [697]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

lm.fit(X_train,y_train)

C= lm.predict(X_val)

display(mean_squared_error(y_val,C))
display(r2_score(y_val,C))
model_dict['OLS'] = [mean_squared_error(y_val,C),r2_score(y_val,C)]

0.004212802187763748

0.8440890478442715

In [698]:
from sklearn.linear_model import Lasso 

lasso_lm = Lasso(alpha=.01)

lasso_lm.fit(X_train,y_train)

D= lasso_lm.predict(X_val)

display(mean_squared_error(y_val,D))
display(r2_score(y_val,D))
model_dict['Lasso'] = [mean_squared_error(y_val,D),r2_score(y_val,D)]

0.008996185450822444

0.6670615478977016

In [699]:
n = 20
for i in range(1,n+1):
    features_for_reducedmodel = list(data.index[:i]) 
    X_train_red = X_train[features_for_reducedmodel]
    X_val_red = X_val[features_for_reducedmodel]
    lm.fit(X_train_red,y_train)
    E = lm.predict(X_val_red)
    model_dict['OLS_reduced_'+str(i)] = [mean_squared_error(y_val,E),r2_score(y_val,E)]

In [700]:
keys = list(model_dict.keys())
values = list(model_dict.values())

performance = pd.DataFrame(data=values, index=keys, columns=metrics)

In [701]:
performance

mse  r2_score
Random_forest   0.003573  0.867773
XGBoost         0.003409  0.873835
OLS             0.004213  0.844089
Lasso           0.008996  0.667062
OLS_reduced_1   0.014243  0.472885
OLS_reduced_2   0.010876  0.597478
OLS_reduced_3   0.010606  0.607470
OLS_reduced_4   0.010576  0.608602
OLS_reduced_5   0.006729  0.750952
OLS_reduced_6   0.006256  0.768477
OLS_reduced_7   0.005298  0.803939
OLS_reduced_8   0.005149  0.809426
OLS_reduced_9   0.005249  0.805730
OLS_reduced_10  0.005109  0.810923
OLS_reduced_11  0.005106  0.811031
OLS_reduced_12  0.004844  0.820713
OLS_reduced_13  0.004794  0.822566
OLS_reduced_14  0.004737  0.824694
OLS_reduced_15  0.004701  0.826009
OLS_reduced_16  0.004700  0.826051
OLS_reduced_17  0.004658  0.827595
OLS_reduced_18  0.004189  0.844952
OLS_reduced_19  0.004178  0.845364
OLS_reduced_20  0.004164  0.845879